In [3]:
import torch

embedding = torch.rand(10, 56,100,100)

In [19]:
# let's get the client and server model from the embedding
from Models.mobilenetv2 import MobileNetV2
from Dataloaders.dataloader_cifar10 import Dataloader_cifar10
import torch.nn as nn
import torch.optim as optim
import torch
from tqdm import tqdm
import os
import sys
import cv2
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

model = MobileNetV2(num_classes=10, weight_path='/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/otherwork/pytorch-cifar/checkpoint/MobileNetV2.pth')
# load weights
client_model = model.get_client_model()
server_model = model.get_server_model()

# split the server model into 2 parts
server_model_1 = server_model[:len(server_model)-1]
server_model_2 = server_model[-1]

# client model and server 1 are fixed, server 2 is trainable
client_model = client_model.train()
server_model_1 = server_model_1.train()
server_model_2 = server_model_2.train()

loss = torch.nn.CrossEntropyLoss()
# https://github.com/d-li14/mobilenetv2.pytorch
# batch size 256; epoch 150; learning rate 0.05; LR decay strategy cosine; weight decay 0.00004
optimizer = torch.optim.Adam(server_model_2.parameters(), lr=0.05)

# put them in cuda
client_model = client_model.cuda()
server_model_1 = server_model_1.cuda()
server_model_2 = server_model_2.cuda()

# get the data
train, test, labels = Dataloader_cifar10(128, 2014)

# also load the original model
# import F
import torch.nn.functional as F
class MobileNetV22(nn.Module):
    # (expansion, out_planes, num_blocks, stride)
    cfg = [(1,  16, 1, 1),
           (6,  24, 2, 1),  # NOTE: change stride 2 -> 1 for CIFAR10
           (6,  32, 3, 2),
           (6,  64, 4, 2),
           (6,  96, 3, 1),
           (6, 160, 3, 2),
           (6, 320, 1, 1)]

    def __init__(self, num_classes=10):
        super(MobileNetV22, self).__init__()
        # NOTE: change conv1 stride 2 -> 1 for CIFAR10
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32)
        self.conv2 = nn.Conv2d(320, 1280, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(1280)
        self.linear = nn.Linear(1280, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for expansion, out_planes, num_blocks, stride in self.cfg:
            strides = [stride] + [1]*(num_blocks-1)
            for stride in strides:
                layers.append(Block(in_planes, out_planes, expansion, stride))
                in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.relu(self.bn2(self.conv2(out)))
        # NOTE: change pooling kernel_size 7 -> 4 for CIFAR10
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
class Block(nn.Module):
    '''expand + depthwise + pointwise'''
    def __init__(self, in_planes, out_planes, expansion, stride):
        super(Block, self).__init__()
        self.stride = stride

        planes = expansion * in_planes
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, groups=planes, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_planes)

        self.shortcut = nn.Sequential()
        if stride == 1 and in_planes != out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_planes),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out + self.shortcut(x) if self.stride==1 else out
        return out
original_model = MobileNetV22()
original_model.load_state_dict(torch.load('/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/MobileNetV2.pth'))
original_model.to('cuda')

# train
# epoch = 20
# for i in tqdm(range(epoch)):
#     for data in train:
#         inputs, labels = data
#         inputs = inputs.cuda()
#         labels = labels.cuda()
#         optimizer.zero_grad()
#         # with torch.no_grad():
#         outputs = client_model(inputs)
#         outputs = server_model_1(outputs)
#         outputs = server_model_2(outputs)
#         # get the index of the max value of the output?
#         loss_train = loss(outputs, labels)
#         loss_train.backward()
#         optimizer.step()
#     print('train loss: ',loss_train.item())

# test
correct = 0
total = 0
with torch.no_grad():
    for data in test:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = client_model(inputs)
        outputs = server_model_1(outputs)
        outputs = server_model_2(outputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # the original model
        outputs2 = original_model(inputs)
        _, predicted2 = torch.max(outputs2.data, 1)

# how did I get the labels?
print('outputs ', outputs)
print('predicted ', predicted)
print('outputs2 ', outputs2)
print('predicted2 ', predicted2)
print('labels ', labels)
print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total)) 


 

/home/tonypeng/anaconda3/envs/iot/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files already downloaded and verified
Files already downloaded and verified


RuntimeError: Error(s) in loading state_dict for MobileNetV22:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layers.0.conv1.weight", "layers.0.bn1.weight", "layers.0.bn1.bias", "layers.0.bn1.running_mean", "layers.0.bn1.running_var", "layers.0.conv2.weight", "layers.0.bn2.weight", "layers.0.bn2.bias", "layers.0.bn2.running_mean", "layers.0.bn2.running_var", "layers.0.conv3.weight", "layers.0.bn3.weight", "layers.0.bn3.bias", "layers.0.bn3.running_mean", "layers.0.bn3.running_var", "layers.0.shortcut.0.weight", "layers.0.shortcut.1.weight", "layers.0.shortcut.1.bias", "layers.0.shortcut.1.running_mean", "layers.0.shortcut.1.running_var", "layers.1.conv1.weight", "layers.1.bn1.weight", "layers.1.bn1.bias", "layers.1.bn1.running_mean", "layers.1.bn1.running_var", "layers.1.conv2.weight", "layers.1.bn2.weight", "layers.1.bn2.bias", "layers.1.bn2.running_mean", "layers.1.bn2.running_var", "layers.1.conv3.weight", "layers.1.b

In [3]:
import importlib
import 
from Models.mobilenetv2 import MobileNetV2
from Dataloaders.dataloader_cifar10 import Dataloader_cifar10
importlib.reload(MobileNetV2)
importlib.reload(Dataloader_cifar10)
model = MobileNetV2(num_classes=10, weight_path='./otherwork/pytorch-cifar/savedpth/cifar10-mobV2-200.pth')


ImportError: module MobileNetV2 not in sys.modules